In [23]:
 %tensorflow_version 2.x
 import tensorflow as tf
 import string
 import requests
 

In [24]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')


In [ ]:
response.text

In [ ]:
data = response.text.split('\n')

In [27]:
data  = " ".join(data)

In [28]:
def clean_text(doc):
  tokens = doc.split()
  table  = str.maketrans('','',string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

In [29]:
tokens  = clean_text(data)
print(tokens[:50])

['this', 'is', 'the', 'etext', 'file', 'presented', 'by', 'project', 'gutenberg', 'and', 'is', 'presented', 'in', 'cooperation', 'with', 'world', 'library', 'inc', 'from', 'their', 'library', 'of', 'the', 'future', 'and', 'shakespeare', 'cdroms', 'project', 'gutenberg', 'often', 'releases', 'etexts', 'that', 'are', 'not', 'placed', 'in', 'the', 'public', 'domain', 'shakespeare', 'this', 'etext', 'has', 'certain', 'copyright', 'implications', 'you', 'should', 'read']


In [30]:
len(tokens)

899788

In [31]:
len(set(tokens))

28113

In [32]:
 length  = 50 + 1

In [33]:
lines  = []
for i in range(length,len(tokens)):
  seq = tokens[i-length:i]
  line  = ' '.join(seq)
  lines.append(line)
  if i > 100000:
    break

print(len(lines))


99951


## Buliding LSTM Model and prepare `x` and `y`

In [34]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [36]:
sequences = np.array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]

In [37]:
vocab_size = len(tokenizer.word_index) + 1

In [38]:
y = to_categorical(y,num_classes=vocab_size)

In [39]:
seq_length = X.shape[1]

##LSTM Model


In [40]:
model = Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
model.add(LSTM(100,return_sequences =True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))



In [41]:
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            450750    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 9015)              910515    
Total params: 1,512,165
Trainable params: 1,512,165
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.fit(X,y,batch_size=256,epochs=200)

Epoch 1/200
391/391 [==============================] - 13s 27ms/step - loss: 6.9432 - accuracy: 0.0280
Epoch 2/200
391/391 [==============================] - 10s 26ms/step - loss: 6.5849 - accuracy: 0.0328
Epoch 3/200
391/391 [==============================] - 10s 26ms/step - loss: 6.4304 - accuracy: 0.0402
Epoch 4/200
391/391 [==============================] - 10s 26ms/step - loss: 6.3126 - accuracy: 0.0453
Epoch 5/200
391/391 [==============================] - 10s 26ms/step - loss: 6.1973 - accuracy: 0.0553
Epoch 6/200
391/391 [==============================] - 10s 26ms/step - loss: 6.0790 - accuracy: 0.0632
Epoch 7/200
391/391 [==============================] - 10s 26ms/step - loss: 5.9722 - accuracy: 0.0696
Epoch 8/200
391/391 [==============================] - 10s 26ms/step - loss: 5.9216 - accuracy: 0.0728
Epoch 9/200
391/391 [==============================] - 10s 25ms/step - loss: 5.8336 - accuracy: 0.0791
Epoch 10/200
391/391 [==============================] - 10s 25ms/step - l

In [46]:
seed_text = lines[123]

In [48]:
def generate_text_seq(model,tokenizer,text_seq_length,seed_text,n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded],maxlen=text_seq_length,truncating='pre')

    y_predic = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predic:
        predicted_word = word
        break
    seed_text = seed_text + ' ' +predicted_word
    text.append(predicted_word)
  return ' '.join(text)


In [ ]:
generate_text_seq(model,tokenizer,seq_length,seed_text,10)